In [106]:
from xml.dom import minidom
import numpy as np
import geopy.distance
import pandas as pd
import xml.etree.ElementTree as ET
import uuid
import matplotlib.pyplot as plt

COLUMNS = ["type", "from", "to", "start_tick", "duration", "slowdown"]
TRIPS_COMPLETE_PATH = "trips_completo.xml"

tree = ET.parse(TRIPS_COMPLETE_PATH)
trips = tree.getroot()

In [107]:
selected_trips = []
edges = {}

In [108]:
df = pd.read_csv('events.csv', sep=';', names=COLUMNS)
forbidden_nodes = []

In [109]:
forbidden_nodes.append(df['from'].unique())
forbidden_nodes.append(df['to'].unique())

In [110]:
forbidden_nodes = np.array(forbidden_nodes).flatten()

In [111]:
def load_nodes(path):
    nodes = []
    events_xml = minidom.parse(path)
    map_nodes_tags = events_xml.getElementsByTagName('node')
    for n in map_nodes_tags:
        nodes.append(int(n.getAttribute('id')))
    return nodes

def load_edges(path):
    map_xml = minidom.parse(path)
    map_links_tags = map_xml.getElementsByTagName('link')
    for n in map_links_tags:
        edges[int(n.getAttribute('id'))] = [
            int(n.getAttribute('from')),
            int(n.getAttribute('to'))
        ]

In [112]:
nodes = load_nodes('map_complete.xml')
load_edges('map_complete.xml')

In [113]:
def find_start_link(origin, destination):
    for k, e in edges.items():
        if (e[0] == origin):
            return k
    return -1

In [114]:
for trip in trips.iter('trip'):
    origin = int(trip.attrib['origin'])
    destination = int(trip.attrib['destination'])
    if (origin in nodes and destination in nodes):
        selected_trips.append(trip)

In [163]:
df = pd.DataFrame(columns=["name", "origin", "destination", "link_origin", "count", "start", "mode", "uuid"])

In [164]:
counter = 0
for s in selected_trips:
    origin = int(s.attrib['origin'])
    destination = int(s.attrib['destination'])
    link_origin = int(find_start_link(origin, destination))
    
    if (link_origin == -1):
        continue
        
    if ((origin in forbidden_nodes) or (destination in forbidden_nodes)):
        continue
    skip_nodes = [303561733, 2006965204, 2465920967, 2397982708, 2397319383]
    if ((origin in skip_nodes) or (destination in skip_nodes)):
        continue
    
    name = s.attrib['name']

    old_count = int(s.attrib['count'])
    start = int(s.attrib['start'])
    mode = s.attrib['mode']
    uuid_trip = 'a'
    df.loc[counter] = [name, origin, destination, link_origin, old_count, start, mode, uuid_trip]
    counter += 1

In [165]:
df = df[(df.start >= 28800) & (df.start <= 30000)]

In [166]:
df.start = df.start - 28800

In [167]:
final_df = pd.DataFrame(columns=["name", "origin", "destination", "link_origin", "count", "start", "mode", "uuid"])    

In [168]:
df.describe()

,name,origin,destination,link_origin,count,start,mode,uuid
count,41,41,41,41,41,41,41,41
unique,41,38,38,38,26,4,1,1
top,2098,303987727,444959652,2150,23,1,car,a
freq,1,4,2,4,8,34,41,41


In [169]:
counter = 0
for row in df.iterrows():
    access = row[1]
    repeat = access['count']
    for v in range(1, repeat):    
        final_df.loc[counter] = [
            counter,
            access['origin'],
            access['destination'],
            access['link_origin'],
            1,
            access['start'],
            access['mode'],
            uuid.uuid4()
        ]
        counter += 1

In [170]:
final_df.to_csv('trips-with-uuid.csv', header=False, sep=';', index=False)

In [171]:
final_df.destination.unique()

array([444959652, 1595406109, 60685603, 661212077, 1389167308, 1828205658,
       1832037287, 4483150089, 303863452, 3316883634, 302805873,
       302805844, 2050908603, 28991286, 60641089, 303561713, 2578181725,
       1832037297, 1117834830, 4484837428, 477363512, 1990934598,
       300058931, 1363061409, 302805865, 303562326, 4167937333,
       2380750803, 165459105, 1942703874, 598665342, 444959901,
       1389167404, 4479246583, 142811324, 3316840685, 303860188, 582142],
      dtype=object)